In [1]:
!gpustat

vclab-gpuserver-57          Sun Mar 24 19:59:53 2024  515.65.01
[0] NVIDIA GeForce RTX 3090 | 64'C,  50 % | 13957 / 24576 MB | xiaoyuzhou(2347M) yuzhenqin(11297M)
[1] NVIDIA GeForce RTX 3090 | 68'C,  42 % | 11442 / 24576 MB | yangli(5157M) xiaoyuzhou(2203M) yangli(3769M)
[2] NVIDIA GeForce RTX 3090 | 65'C,  13 % | 16073 / 24576 MB | yuzhenqin(11993M) yangli(3769M)
[3] NVIDIA GeForce RTX 3090 | 67'C,   1 % | 15251 / 24576 MB | yuzhenqin(11171M) yangli(3769M)
[4] NVIDIA GeForce RTX 3090 | 54'C,  22 % | 14074 / 24576 MB | robotics(13763M)
[5] NVIDIA GeForce RTX 3090 | 58'C,  13 % | 14374 / 24576 MB | robotics(14063M)
[6] NVIDIA GeForce RTX 3090 | 50'C,  21 % | 14374 / 24576 MB | robotics(14063M)
[7] NVIDIA GeForce RTX 3090 | 54'C,  19 % | 14374 / 24576 MB | robotics(14063M)


In [2]:
# 在之前的蒸馏基线上，加入ABCHLN单导联的GAN，生成4倍的数据，在ABCHLN验证，在TR进行测试。

In [3]:
from models.DG_models import generate_model
from datasets import train_loader,val_loader,test_loader
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] ='5'
device='cuda:0'
import wfdb
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from scipy import signal
from tqdm import tqdm
import math
import time
import torch
from torch import nn
from torchsummary import summary
import sklearn.metrics
import csv
import copy
from utils import *
import torch.nn.functional as F  
from collections import deque  
import random


import sys
import ast
import csv
import math
import time
import wfdb
import copy
import torch
import random
import numpy as np
import pandas as pd
from utils import *
import torch.nn as nn
from tqdm import tqdm
import sklearn.metrics
from scipy import signal
import torch.nn.init as init
from collections import deque  
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchsummary import summary
import torch.autograd as autograd
from torch.autograd import Variable
from collections import defaultdict
import torch.backends.cudnn as cudnn
from torch.optim import lr_scheduler
from torch.nn import functional as F
from models.DG_models import generate_model
from datasets import train_loader,val_loader,test_loader
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer


In [4]:
base_lr=2e-3
gan_aug=3
batch_size=128
max_lr=base_lr*batch_size/256
min_lr=max_lr/100
distill=True
DG_method_student='DG_GR'
DG_method_teacher='teacher_self_distill_leadI_GRL'
GRL_loss_beta=0.05
DGGR_smooth_eps=0.3
ts_label_beta=0.3
ts_feature_beta=0.3
train_val_test_split=[0.8,0.2,1] # 1 for inter
base_optimizer=torch.optim.Adam
loss_fn_clf=nn.CrossEntropyLoss()
base_model_student='cnn_Ag'
base_model_teacher='cnn_Ag_teacher_self_distill_leadI'
class_num=3
smote=False
pair_num_per_anchor_batch=5
CTL_margin=0.1
warm_epoch=2
total_epoch=30
CRL_queue_len=5
# 计算两个模型标签的余弦相似度损失

class CosineSimilarityLoss(torch.nn.Module):  
    def __init__(self):  
        super(CosineSimilarityLoss, self).__init__()  
  
    def forward(self, input1, input2):  
        cos_sim = torch.mean(F.cosine_similarity(input1, input2))
        loss = 1 - cos_sim  
        return loss

loss_fn_tslb = CosineSimilarityLoss()

# 基于欧氏距离的特征对比损失 

# 没有温度系数没考虑，因为relu已经考虑了平滑的效果。

# 如果想用温度系数建议去问大模型。
  
class ContrastiveLoss(torch.nn.Module):  
    def __init__(self, margin=1.):  
        super(ContrastiveLoss, self).__init__()  
        self.margin = torch.tensor(margin, dtype=torch.float32)
  
    def forward(self, anchor, positive, negative):  
#         print('anchor:',anchor)
#         print('positive:',positive)
#         print('negative:',negative)
        # 计算锚点和正样本之间的余弦相似度  
        similarity_anchor_positive = F.cosine_similarity(anchor, positive, dim=-1)  
#         print('positive:',similarity_anchor_positive)
        # 计算锚点和负样本之间的余弦相似度  
        similarity_anchor_negative = F.cosine_similarity(anchor, negative, dim=-1)  
#         print('negtive:',similarity_anchor_negative)
  
        # 计算损失  
        loss = torch.mean(F.relu(similarity_anchor_positive-similarity_anchor_negative))
#         print('cft_loss=',loss)
        return loss

feature_contrastive_loss_fn = ContrastiveLoss(margin=CTL_margin)

# 维护教师的特征内存队列和标签队列

class TeacherQueue:  
    def __init__(self, max_size):  
        self.max_size = max_size  
        self.feat_queue = deque(maxlen=max_size)  
        self.label_queue = deque(maxlen=max_size)  
  
    def enqueue(self, features,labels):  
        """  
        将特征向量加入队列  
        :param features: 特征向量，形状为[batch_size, feature_dim]  
        """  
        if len(self.feat_queue) == self.max_size:  
            self.dequeue()  # 如果队列已满，先出队  
        self.feat_queue.append(features)  
        self.label_queue.append(labels)  
  
    def dequeue(self):  
        """  
        从队列中移除最早加入的特征向量，无需返回值。  
        """  
        self.feat_queue.popleft()  
        self.label_queue.popleft()  
        
        return 0
    
    def get_queue_len(self):
        return len(self.feat_queue)
  
    def get_all_queue(self):  
        """  
        返回队列中所有的特征向量和标签，形状为[total_size, feature_dim],[total_size, label_dim]  
        """  
        return torch.cat(list(self.feat_queue)),torch.cat(list(self.label_queue))

def get_class_f1(tar,pred):
    # 多分类多标签或单标签，输入为概率或one hot
    pred_idx = pred.argmax(axis=1)
    pred_=(pred_idx[:,None] == np.arange(pred.shape[1])).astype(int)
    F1_normal_score,F1_AF_score,F1_other_score=list(sklearn.metrics.f1_score(tar, pred_, average=None))
    return F1_normal_score,F1_AF_score,F1_other_score

def val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          test_datasloader, device):
    
    student_model.eval() 
    running_loss = None 
    test_pred=0
    running_loss = []
    test_tar=0
    for step, (in_data_, targets_) in (enumerate(test_datasloader)):
        tmp_data=np.zeros((in_data_.shape[0],1000,2))
        tmp_data[:,:,0]=in_data_[:,:,0]
        tmp_data[:,:-1,1]=in_data_[:,1:,0]-in_data_[:,0:-1,0]
        preds_,_,_=student_model(torch.tensor(tmp_data).float().to(device))
        preds_=preds_.float()
        targets_for_loss=torch.max(targets_[:,:class_num],1).indices.long().to(device)
        loss=loss_fn_clf(preds_,targets_for_loss)    
        running_loss.append(loss.item())
        mean_loss=np.mean(running_loss)

        # 使用AUC作为评价指标
        if type(test_pred)==int:
            test_pred=preds_.clone().detach().cpu().numpy()
        else:
            test_pred=np.concatenate((test_pred,
                                preds_.clone().detach().cpu().numpy(),
                                ),axis=0)
        if type(test_tar)==int:
            test_tar=targets_.clone().detach().cpu().numpy()
        else:
            test_tar=np.concatenate((test_tar,
                                targets_.clone().detach().cpu().numpy(),
                                ),axis=0)

    print('epoch=',epoch,)
    print("val_loss=",mean_loss)
    print("________________________________")
    mean_macro_auc=sklearn.metrics.roc_auc_score(test_tar, test_pred, average='macro')
#     print('test mean_macro_auc= ',mean_macro_auc)
    
#     print(test_tar.shape)
#     print(test_pred.shape)
    
    F1_score=get_macro_f1(test_tar,test_pred)
#     print('test F1_score= ',F1_score)
    
    precision=get_macro_precision(test_tar,test_pred)
#     print('test precision= ',precision)
    
    recall=get_macro_recall(test_tar,test_pred)
#     print('test recall= ',recall)

    acc=get_acc(test_tar,test_pred)
#     print('test acc= ',acc)
    
    F1_normal_score,F1_AF_score,F1_other_score=get_class_f1(test_tar, test_pred)
    return str(int(round(1000*mean_macro_auc))),str(int(round(1000*precision))),str(int(round(1000*recall))),str(int(round(1000*F1_score))),str(int(round(1000*F1_normal_score))),str(int(round(1000*F1_AF_score))),str(int(round(1000*F1_other_score))),str(int(round(1000*acc)))

def test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          test_datasloader, device):
    
    student_model.eval() 
    running_loss = None 
    test_pred=0
    running_loss = []
    test_tar=0
    for step, (in_data_, targets_) in (enumerate(test_datasloader)):
        tmp_data=np.zeros((in_data_.shape[0],1000,2))
        tmp_data[:,:,0]=in_data_[:,:,0]
        tmp_data[:,:-1,1]=in_data_[:,1:,0]-in_data_[:,0:-1,0]
        preds_,_,_=student_model(torch.tensor(tmp_data).float().to(device))
        preds_=preds_.float()
        targets_for_loss=torch.max(targets_[:,:class_num],1).indices.long().to(device)
        loss=loss_fn_clf(preds_,targets_for_loss)    
        running_loss.append(loss.item())
        mean_loss=np.mean(running_loss)

        # 使用AUC作为评价指标
        if type(test_pred)==int:
            test_pred=preds_.clone().detach().cpu().numpy()
        else:
            test_pred=np.concatenate((test_pred,
                                preds_.clone().detach().cpu().numpy(),
                                ),axis=0)
        if type(test_tar)==int:
            test_tar=targets_.clone().detach().cpu().numpy()
        else:
            test_tar=np.concatenate((test_tar,
                                targets_.clone().detach().cpu().numpy(),
                                ),axis=0)

    print('epoch=',epoch,)
    print("test_loss=",mean_loss)
    print("________________________________")
    mean_macro_auc=sklearn.metrics.roc_auc_score(test_tar, test_pred, average='macro')
#     print('test mean_macro_auc= ',mean_macro_auc)
    
#     print(test_tar.shape)
#     print(test_pred.shape)
    
    F1_score=get_macro_f1(test_tar,test_pred)
#     print('test F1_score= ',F1_score)
    
    precision=get_macro_precision(test_tar,test_pred)
#     print('test precision= ',precision)
    
    recall=get_macro_recall(test_tar,test_pred)
#     print('test recall= ',recall)

    acc=get_acc(test_tar,test_pred)
#     print('test acc= ',acc)
    
    F1_normal_score,F1_AF_score,F1_other_score=get_class_f1(test_tar, test_pred)
    return str(int(round(1000*mean_macro_auc))),str(int(round(1000*precision))),str(int(round(1000*recall))),str(int(round(1000*F1_score))),str(int(round(1000*F1_normal_score))),str(int(round(1000*F1_AF_score))),str(int(round(1000*F1_other_score))),str(int(round(1000*acc)))

tem_relu=nn.ReLU(inplace=False)

# 定义训练流程
def train_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_datasloader,device,GRL_loss_beta,GAN_aug=None,GAN_model=None):
    student_model.train()
    running_loss = None 
    # 维护一个对比损失的内存队列
    for step, (in_data_, targets_) in (enumerate(tqdm(train_datasloader))):
        # 教师模型的预测
        
        assert (gan_aug==3) # 进行一次教师模型的前向传播和队列存储，一次原有单导联蒸馏，3次单导联生成蒸馏
        
        teacher_diag_preds_,_,_,teacher_feature,_,_,_,_=teacher_model(in_data_.float().to(device))
        teacher_feature=torch.flatten(teacher_feature,1).detach()
#         teacher_feature=tem_relu(teacher_feature).detach()
#         print(teacher_feature.shape)
        # 学生模型的预测
        current_iter=step+epoch*len(train_datasloader)
        warm_iter=warm_epoch*len(train_datasloader)
        total_iter=total_epoch*len(train_datasloader)
        adjust_learning_rate(optimizer,current_iter,warm_iter,total_iter,max_lr,min_lr)
        
        
#         with torch.no_grad():
#             in_data_1 = in_data_[:,:,:1].float().to(device).clone()
#             in_data_2 = GAN_model.generate_images(torch.cat((in_data_1,in_data_1,in_data_1),dim=0))  # 32,

#             in_data_GAN4=torch.cat((in_data_1,in_data_2),dim=0)
                                
        optimizer.zero_grad()
        
        
        
        tmp_data=np.zeros((in_data_.shape[0],1000,2))
        tmp_data[:,:,0]=in_data_[:,:,0]
        tmp_data[:,:-1,1]=in_data_[:,1:,0]-in_data_[:,0:-1,0]
        student_diag_preds_,student_domain_preds_,student_feature=student_model(torch.tensor(tmp_data).float().to(device))
        student_diag_preds_=student_diag_preds_.float()
        student_domain_preds_=student_domain_preds_.float()
        teacher_feature=teacher_feature.float()
        current_label=targets_[:,:class_num].clone()
        Teacher_queue.enqueue(teacher_feature,current_label)
        student_feature=torch.flatten(student_feature,1).float()

        # 损失函数的计算和反向传播
        # 分类损失部分
        targets_for_diag_loss=torch.max(targets_[:,:class_num],1).indices.long().to(device)  # 这里的2是指两类。
        loss_diag=loss_fn_clf(student_diag_preds_,targets_for_diag_loss)
        # domain损失部分
        targets_for_domain_loss=torch.max(targets_[:,class_num+1:],1).indices.long().to(device)
                            # 这里的3是指后面的领域情况。
        loss_domain=loss_fn_clf(student_domain_preds_,targets_for_domain_loss)
#         # 分类结果差异损失部分
        loss_tslb = loss_fn_tslb(teacher_diag_preds_.detach(), student_diag_preds_)  
#         # 特征对比损失部分
#         # 获取教师模型保存的特征向量
        if (Teacher_queue.get_queue_len()==0):
            pass
        else:
            all_teacher_feature,all_teacher_labels=Teacher_queue.get_all_queue()
            if  torch.allclose(all_teacher_feature, all_teacher_feature[0]) : # 队列中只有一类标签  
                pass
            else:
#                 print('loss_tsft:',1)
                feature_contrastive_loss_list=[]
                p_teacher_set=torch.zeros(pair_num_per_anchor_batch,current_label.shape[0],student_feature.shape[-1]).to(device)
                n_teacher_set=torch.zeros(pair_num_per_anchor_batch,current_label.shape[0],student_feature.shape[-1]).to(device)

#                 student_feature
#                 current_label
                for tmp_stu_idx in range(current_label.shape[0]):
                    # 获取正样本索引元组
#                     print('current_label[tmp_stu_idx]',current_label[tmp_stu_idx])
#                     print('all_teacher_labels',all_teacher_labels)
                    positive_idxs= [u for u, label in enumerate(all_teacher_labels[:, 0]+2*all_teacher_labels[:, 1]) \
                                     if label == (current_label[tmp_stu_idx,0].item()+2*current_label[tmp_stu_idx,1].item())]
                    
#                     print(positive_idxs)
#                     for u,label in enumerate(all_teacher_labels[:, 0]+2*all_teacher_labels[:, 1]):
#                         print( u,label )
#                         print(current_label[tmp_stu_idx,0].item()+2*current_label[tmp_stu_idx,1].item())
#                     print(current_label[tmp_stu_idx,0].item()+2*current_label[tmp_stu_idx,1].item())
#                         print('_______________________________________________')
                    
#                     print('positive_idxs',positive_idxs)
                    # 获取负样本索引元组
                    negative_idxs=[x for x in range(all_teacher_labels.shape[0]) if x not in positive_idxs]  
#                     print('negative_idxs',negative_idxs)
                    for iii in range(pair_num_per_anchor_batch):
                        p_teacher_set[iii,tmp_stu_idx,:]=all_teacher_feature[random.choice(positive_idxs)]
                        n_teacher_set[iii,tmp_stu_idx,:]=all_teacher_feature[random.choice(negative_idxs)]
                for iii in range(pair_num_per_anchor_batch):
                    feature_contrastive_loss_list.append(feature_contrastive_loss_fn(student_feature, p_teacher_set[iii], n_teacher_set[iii]))
                loss_tsft=torch.mean(torch.tensor(feature_contrastive_loss_list))        
                        


        # 求和
#         print(loss_diag)
#         print(loss_domain)
#         print(loss_tslb)
#         print(loss_tsft)
#         print('___________________')
#         loss=loss_diag+GRL_loss_beta*loss_domain+ts_label_beta*loss_tslb
        if (loss_tsft):
            loss=loss_diag+GRL_loss_beta*loss_domain+ts_label_beta*loss_tslb+ts_feature_beta*loss_tsft
        else:
            loss=loss_diag+GRL_loss_beta*loss_domain+ts_label_beta*loss_tslb
        # 反向传播
        loss.backward()
        optimizer.step()
        if running_loss is None:
            running_loss = loss.item() # 只输出重构损失
        else:
            running_loss = running_loss * .9 + loss.item() * .1   
            
            
            
            
        for gan_i in range(gan_aug):
            
            with torch.no_grad():
                in_data_1 = in_data_[:,:,:1].float().to(device).clone()
                in_data_gan = GAN_model.generate_images(in_data_1).cpu().numpy()  # 32,

            optimizer.zero_grad()
            tmp_data=np.zeros((in_data_gan.shape[0],1000,2))
            tmp_data[:,:,0]=in_data_gan[:,:,0]
            tmp_data[:,:-1,1]=in_data_gan[:,1:,0]-in_data_gan[:,0:-1,0]
            student_diag_preds_,student_domain_preds_,student_feature=student_model(torch.tensor(tmp_data).float().to(device))
            student_diag_preds_=student_diag_preds_.float()
            student_domain_preds_=student_domain_preds_.float()
            student_feature=torch.flatten(student_feature,1).float()

            # 损失函数的计算和反向传播
            # 分类损失部分
            targets_for_diag_loss=torch.max(targets_[:,:class_num],1).indices.long().to(device)  # 这里的2是指两类。
            loss_diag=loss_fn_clf(student_diag_preds_,targets_for_diag_loss)
            # domain损失部分
            targets_for_domain_loss=torch.max(targets_[:,class_num+1:],1).indices.long().to(device)
                                # 这里的3是指后面的领域情况。
            loss_domain=loss_fn_clf(student_domain_preds_,targets_for_domain_loss)
    #         # 分类结果差异损失部分
            loss_tslb = loss_fn_tslb(teacher_diag_preds_.detach(), student_diag_preds_)  
    #         # 特征对比损失部分
    #         # 获取教师模型保存的特征向量
            if (Teacher_queue.get_queue_len()==0):
                pass
            else:
                all_teacher_feature,all_teacher_labels=Teacher_queue.get_all_queue()
                if  torch.allclose(all_teacher_feature, all_teacher_feature[0]) : # 队列中只有一类标签  
                    pass
                else:
    #                 print('loss_tsft:',1)
                    feature_contrastive_loss_list=[]
                    p_teacher_set=torch.zeros(pair_num_per_anchor_batch,current_label.shape[0],student_feature.shape[-1]).to(device)
                    n_teacher_set=torch.zeros(pair_num_per_anchor_batch,current_label.shape[0],student_feature.shape[-1]).to(device)

    #                 student_feature
    #                 current_label
                    for tmp_stu_idx in range(current_label.shape[0]):
                        # 获取正样本索引元组
    #                     print('current_label[tmp_stu_idx]',current_label[tmp_stu_idx])
    #                     print('all_teacher_labels',all_teacher_labels)
                        positive_idxs= [u for u, label in enumerate(all_teacher_labels[:, 0]+2*all_teacher_labels[:, 1]) \
                                         if label == (current_label[tmp_stu_idx,0].item()+2*current_label[tmp_stu_idx,1].item())]

    #                     print(positive_idxs)
    #                     for u,label in enumerate(all_teacher_labels[:, 0]+2*all_teacher_labels[:, 1]):
    #                         print( u,label )
    #                         print(current_label[tmp_stu_idx,0].item()+2*current_label[tmp_stu_idx,1].item())
    #                     print(current_label[tmp_stu_idx,0].item()+2*current_label[tmp_stu_idx,1].item())
    #                         print('_______________________________________________')

    #                     print('positive_idxs',positive_idxs)
                        # 获取负样本索引元组
                        negative_idxs=[x for x in range(all_teacher_labels.shape[0]) if x not in positive_idxs]  
    #                     print('negative_idxs',negative_idxs)
                        for iii in range(pair_num_per_anchor_batch):
                            p_teacher_set[iii,tmp_stu_idx,:]=all_teacher_feature[random.choice(positive_idxs)]
                            n_teacher_set[iii,tmp_stu_idx,:]=all_teacher_feature[random.choice(negative_idxs)]
                    for iii in range(pair_num_per_anchor_batch):
                        feature_contrastive_loss_list.append(feature_contrastive_loss_fn(student_feature, p_teacher_set[iii], n_teacher_set[iii]))
                    loss_tsft=torch.mean(torch.tensor(feature_contrastive_loss_list))        



            # 求和
    #         print(loss_diag)
    #         print(loss_domain)
    #         print(loss_tslb)
    #         print(loss_tsft)
    #         print('___________________')
    #         loss=loss_diag+GRL_loss_beta*loss_domain+ts_label_beta*loss_tslb
            if (loss_tsft):
                loss=loss_diag+GRL_loss_beta*loss_domain+ts_label_beta*loss_tslb+ts_feature_beta*loss_tsft
            else:
                loss=loss_diag+GRL_loss_beta*loss_domain+ts_label_beta*loss_tslb
            # 反向传播
            loss.backward()
            optimizer.step()
            if running_loss is None:
                running_loss = loss.item() # 只输出重构损失
            else:
                running_loss = running_loss * .9 + loss.item() * .1   
            
            
            
            
            
            
            
            
            
            
    print('epoch=',epoch,)
    print("train_running_loss=",running_loss)

            # loss的打印
            
    # 特征对比损失部分    
    # 分类结果差异损失部分
    # domain损失部分
    # 分类损失部分





In [5]:
# 生成模型定义，包括两个encoder和一个decoder

class AdaptiveInstanceNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(AdaptiveInstanceNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        # weight and bias are dynamically assigned
        self.weight = None
        self.bias = None
        # just dummy buffers, not used
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        assert self.weight is not None and self.bias is not None, "Please assign weight and bias before calling AdaIN!"
        b, c = x.size(0), x.size(1)
        running_mean = self.running_mean.repeat(b)
        running_var = self.running_var.repeat(b)

        # Apply instance norm
        x_reshaped = x.contiguous().view(1, b * c, *x.size()[2:])
        out = F.batch_norm(
            x_reshaped, running_mean, running_var, self.weight, self.bias,
            True, self.momentum, self.eps)

        return out.view(b, c, *x.size()[2:])

    def __repr__(self):
        return self.__class__.__name__ + '(' + str(self.num_features) + ')'

    
class Decoder(nn.Module):
    def __init__(self,):
        super(Decoder, self).__init__()

        # dec
        self.conv1=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn1=AdaptiveInstanceNorm2d(64)
        self.ac1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv2=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn2=AdaptiveInstanceNorm2d(64)
        self.ac2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv3=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn3=AdaptiveInstanceNorm2d(64)
        self.ac3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        
        
        self.conv4=nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.bn4=AdaptiveInstanceNorm2d(32)
        self.ac4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv5=nn.Conv1d(32, 32, kernel_size=3, padding=1)
        self.bn5=AdaptiveInstanceNorm2d(32)
        self.ac5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv6=nn.Conv1d(32, 16, kernel_size=3, padding=1)
        self.bn6=AdaptiveInstanceNorm2d(16)
        self.ac6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv7=nn.Conv1d(16, 16, kernel_size=3, padding=1)
        self.bn7=AdaptiveInstanceNorm2d(16)
        self.ac7=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv8=nn.Conv1d(16, 8, kernel_size=3, padding=1)
        self.bn8=AdaptiveInstanceNorm2d(8)
        self.ac8=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv9=nn.Conv1d(8, 8, kernel_size=3, padding=1)
        self.ac9=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv10=nn.Conv1d(8, 1, kernel_size=1)
        
    def forward(self, x):
        
        x=self.ac1(x+self.bn1(self.conv1(x)))
        x=self.ac2(x+self.bn2(self.conv2(x)))
        x=self.ac3(x+self.bn3(self.conv3(x)))
        x=self.ac4(self.bn4(self.conv4(x)))
        x=self.ac5(self.bn5(self.conv5(x)))
        x=self.ac6(self.bn6(self.conv6(x)))
        x=self.ac7(self.bn7(self.conv7(x)))
        x=self.ac8(self.bn8(self.conv8(x)))
        x=self.conv10(self.ac9(self.conv9(x)))
        
        return x


class ECG_gen(nn.Module):  
    def __init__(self,):  
        super(ECG_gen, self).__init__()  
        
        # cont_enc
        
        self.conv1=nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.bn1=nn.InstanceNorm1d(8)
        self.ac1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv2=nn.Conv1d(8, 8, kernel_size=3, padding=1)
        self.bn2=nn.InstanceNorm1d(8)
        self.ac2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv3=nn.Conv1d(8, 16, kernel_size=3, padding=1)
        self.bn3=nn.InstanceNorm1d(16)
        self.ac3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv4=nn.Conv1d(16, 16, kernel_size=3, padding=1)
        self.bn4=nn.InstanceNorm1d(16)
        self.ac4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv5=nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.bn5=nn.InstanceNorm1d(32)
        self.ac5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv6=nn.Conv1d(32, 32, kernel_size=3, padding=1)
        self.bn6=nn.InstanceNorm1d(32)
        self.ac6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        self.conv7=nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn7=nn.InstanceNorm1d(64)
        self.ac7=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv8=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn8=nn.InstanceNorm1d(64)
        self.ac8=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        
        
        self.conv9=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn9=nn.InstanceNorm1d(64)
        self.ac9=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv10=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn10=nn.InstanceNorm1d(64)
        self.ac10=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv11=nn.Conv1d(64, 64, kernel_size=3, padding=1)
        self.bn11=nn.InstanceNorm1d(64)
        self.ac11=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        
        # style_enc
        
        self.conv_sty_1=nn.Conv1d(1, 8, kernel_size=3, padding=1)
        self.ac_sty_1=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_2=nn.Conv1d(8, 16, kernel_size=3, padding=1)
        self.ac_sty_2=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_3=nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.ac_sty_3=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_4=nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.ac_sty_4=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_5=nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.ac_sty_5=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.conv_sty_6=nn.Conv1d(128, 256, kernel_size=3, padding=1)
        self.ac_sty_6=nn.LeakyReLU(0.01, inplace=True)  # Replace ReLU with LeakyReLU
        self.pool_sty=nn.AdaptiveAvgPool1d(1)
        self.fc_sty=nn.Conv1d(256, 8, kernel_size=1, padding=0)
        # bs,8,1
        
        self.dec=Decoder()
        
        self.mlp_1=nn.Linear(8, 256, bias=True)
        self.mlp_2=nn.LeakyReLU(0.01, inplace=True)
        self.mlp_3=nn.Linear(256, 256, bias=True)
        self.mlp_4=nn.LeakyReLU(0.01, inplace=True)
        self.mlp_5=nn.Linear(256, self.get_num_adain_params(self.dec), bias=True)
        
        
    def encode(self, x):  # bs,1000,1  x1是特征，x2是风格
        
        x=x.transpose(-1,-2) # bs,1,1000
        
        xc=self.conv1(x)
        xc=self.bn1(xc)
        xc=self.ac1(xc)
        xc=self.ac2(self.bn2(self.conv2(xc)))
        xc=self.ac2(self.bn2(self.conv2(xc)))
        xc=self.ac3(self.bn3(self.conv3(xc)))
        xc=self.ac4(self.bn4(self.conv4(xc)))
        xc=self.ac5(self.bn5(self.conv5(xc)))
        xc=self.ac6(self.bn6(self.conv6(xc)))
        xc=self.ac7(self.bn7(self.conv7(xc)))
        xc=self.ac8(self.bn8(self.conv8(xc)))
        xc=self.ac9(xc+self.bn9(self.conv9(xc)))
        xc=self.ac10(xc+self.bn10(self.conv10(xc)))
        content=self.ac11(xc+self.bn11(self.conv11(xc)))   # bs,64,1000

        
        
        xs=self.conv_sty_1(x)
        xs=self.ac_sty_1(xs)
        xs=self.conv_sty_2(xs)
        xs=self.ac_sty_2(xs)
        xs=self.conv_sty_3(xs)
        xs=self.ac_sty_3(xs)
        xs=self.conv_sty_4(xs)
        xs=self.ac_sty_4(xs)
        xs=self.conv_sty_5(xs)
        xs=self.ac_sty_5(xs)
        xs=self.conv_sty_6(xs)
        xs=self.ac_sty_6(xs)
        xs=self.pool_sty(xs)
        style=self.fc_sty(xs)
        
        return content, style
    
    def decode(self,content, style):  # x1逆向操作，x2应用到自适应归一化层
        
        adain_params = self.mlp_1(style.view(style.size(0), -1))
        adain_params = self.mlp_2(adain_params)
        adain_params = self.mlp_3(adain_params)
        adain_params = self.mlp_4(adain_params)
        adain_params = self.mlp_5(adain_params)
        self.assign_adain_params(adain_params, self.dec)
        x=self.dec(content)
        x=x.transpose(-1,-2)
        
        return x
        
        
    def assign_adain_params(self, adain_params, model):
        for m in model.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                mean = adain_params[:, :m.num_features]
                std = adain_params[:, m.num_features:2*m.num_features]
                m.bias = mean.contiguous().view(-1)
                m.weight = std.contiguous().view(-1)
                if adain_params.size(1) > 2*m.num_features:
                    adain_params = adain_params[:, 2*m.num_features:]
        
        
    def get_num_adain_params(self,model):
        num_adain_params = 0
        for m in model.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                num_adain_params += 2*m.num_features
        return num_adain_params
        
    def forward(self, images):
        # reconstruct an image
        content, style_fake = self.encode(images)
        images_recon = self.decode(content, style_fake)
        return images_recon
    
    
    
# 加载模型的流程定义

def load_munit_model(model_path):

    return MUNITModelOfNatVar(model_path).to(device)



class MUNITModelOfNatVar(nn.Module):
    def __init__(self, fname: str):

        super(MUNITModelOfNatVar, self).__init__()

        self._fname = fname
        self._gen_A, self._gen_B = self.__load()
        self.delta_dim = 8

    def forward(self, x, delta):

        orig_content, _ = self._gen_A.encode(x)
        orig_content = orig_content.clone().detach().requires_grad_(False)
        x_out = self._gen_B.decode(orig_content, delta)

        return x_out

    def __load(self):
        """Load MUNIT model from file."""

        def load_munit(fname, letter):
            gen = ECG_gen()
            gen.load_state_dict(torch.load(fname)[letter])
            return gen.eval()

        gen_A = load_munit(self._fname, 'a')
        gen_B = load_munit(self._fname, 'b')

        return gen_A, gen_B     # original order
        
# 算法总模型定义


class Algorithm(torch.nn.Module):

    def __init__(self,):
        super(Algorithm, self).__init__()

    def update(self, in_data_, targets_, unlabeled=None):
        raise NotImplementedError

    def predict(self, x):
        raise NotImplementedError

class ERM(Algorithm):

    def __init__(self, ):
        super(ERM, self).__init__()


        self.network = generate_model(base_model='cnn_Ag',input_channels=1, num_classes=3,DG_method=None,domain_classes=6,distill=False)
        self.optimizer = torch.optim.Adam(self.network.parameters(),lr=1e-3,weight_decay=0)

    def update(self, in_data_, targets_, unlabeled=None):
        
        loss = F.cross_entropy(self.predict(in_data_), targets_)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return {'loss': loss.item()}

    def predict(self, x):
        return self.network(x)

class MBDG_Base(ERM):
    def __init__(self, ):
        super(MBDG_Base, self).__init__()
    
        self.G = load_munit_model('save_model_states/ABCLN_H_/gen_00600000.pt')

    @staticmethod
    def kl_div(dist1, dist2):
        return F.kl_div(torch.log(dist2), dist1, reduction='batchmean')

    def predict(self, x):
        return self.network(x)

    @torch.no_grad()
    def generate_images(self, images):
        delta = torch.randn(images.size(0), self.G.delta_dim).to(device).requires_grad_(False)
        return self.G(images, delta)

    def calc_dist_reg(self, x, clean_output):
        mb_images = self.generate_images(x)
        mb_output = F.softmax(self.predict(mb_images), dim=1)
        return self.kl_div(F.softmax(clean_output, dim=1), mb_output)

    @staticmethod
    def relu(x):
        return x if x > 0 else torch.tensor(0).to(device)

class MBDG(MBDG_Base):

    def __init__(self, ):
        super(MBDG, self).__init__()
        self.dual_var = torch.tensor(1.0).to(device).requires_grad_(False)

    def update(self, in_data_, targets_, unlabeled=None):

        clean_output = self.predict(in_data_)
        clean_loss = F.cross_entropy(clean_output, targets_)
        dist_reg = self.calc_dist_reg(in_data_, clean_output)

        loss = clean_loss + self.dual_var * dist_reg

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        const_unsat = dist_reg.detach() - 0.025
        self.dual_var = self.relu(self.dual_var + 0.05 * const_unsat)

        return {'loss': loss.item(), 'dist_reg': dist_reg.item(), 'dual_var': self.dual_var.item()}

    
    
cls_model=MBDG().to(device)   
cls_model.eval()

base_model=cnn_Ag


MBDG(
  (network): CNN_Ag(
    (conv1): Conv1d_(1, 32, kernel_size=(16,), stride=(1,), bias=False)
    (res_layers): Sequential(
      (0): Block_Ag(
        (maxpooling): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
        (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv1): Conv1d_(32, 32, kernel_size=(16,), stride=(1,))
        (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (conv2): Conv1d_(32, 32, kernel_size=(16,), stride=(1,))
      )
      (1): Block_Ag(
        (maxpooling): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv1): Conv1d_(32, 32, kernel_size=(16,), stride=(2,))
        (bn2): BatchNorm1d(3

In [6]:
# 加载数据
train_data_list=['L_12','B_12','C_12','A_12','N_12']

train_datasloader_instance=train_loader(dataname=train_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,smote=smote,maple=False,class_num=class_num)
train_datasloader=train_datasloader_instance.loader()
 
val_data_list_A=['A_12']
val_data_list_B=['B_12']
val_data_list_C=['C_12']
val_data_list_N=['N_12']
val_data_list_L=['L_12']
# val_data_list_N=['N_12']


val_datasloader_A=val_loader(dataname=val_data_list_A,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_B=val_loader(dataname=val_data_list_B,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_C=val_loader(dataname=val_data_list_C,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_N=val_loader(dataname=val_data_list_N,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_L=val_loader(dataname=val_data_list_L,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
# val_datasloader_N=val_loader(dataname=val_data_list_N,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()



test_H_data_list=['H']
test_H_datasloader=test_loader(dataname=test_H_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()

# test_R_data_list=['R_3cls']
# test_R_datasloader=test_loader(dataname=test_R_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()





train data L_12
train data size : 16824
train label class :  [13511.  1158.  2155. 16824.     0.     0.     0.     0.]
train data B_12
train data size : 5501
train label class :  [ 742.  859. 3900.    0. 5501.    0.    0.    0.]
train data C_12
train data size : 24194
train label class :  [ 6719.  2039. 15436.     0.     0. 24194.     0.     0.]
train data A_12
train data size : 8274
train label class :  [1393.  480. 6401.    0.    0.    0. 8274.    0.]
train data N_12
train data size : 8516
train label class :  [1462. 1769. 5285.    0.    0.    0.    0. 8516.]
total train data shape =  (63309, 1000, 12)
total train label shape =  (63309, 8)
total train data class =  [23827.  6305. 33177. 16824.  5501. 24194.  8274.  8516.]
val data A_12
val data size : 2069
val label class :  [ 359.   90. 1620.]
total val data shape =  (2069, 1000, 12)
total val label shape =  (2069, 3)
total val data class =  [ 359.   90. 1620.]
val data B_12
val data size : 1376
val label class :  [176. 239. 961.]
t

In [7]:
# 更改加载的教师模型字典：
# 加载5项不同的教师模型
# 开始训练流程
Teacher_queue=TeacherQueue(max_size=CRL_queue_len)
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model=generate_model(base_model=base_model_teacher,input_channels=12, num_classes=class_num,DG_method=DG_method_teacher,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model.load_state_dict(torch.load('/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/1711095663_ABCLN_H_proposed_cnnAg_zong_f1.pth'))
teacher_model.eval()
print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)

mdst_path='/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/'
mdst_name=str(int(time.time()))+'_ABCLN_H_zishiying_proposed_xuesheng_cnnAg_zong_f1.pth'

max_val_f1_all=0
max_epoch=0
max_test_H_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}
# max_test_R_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}

for epoch in range(total_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    train_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_datasloader,device,GRL_loss_beta,GAN_aug='0211',GAN_model=cls_model)
    
    auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_A, device)
    print(auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc)
    auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_B, device)
    print(auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc)
    auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_C, device)
    print(auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc)
    auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_N, device)
    print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
    auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_L, device)
    print(auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc)
#     auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           val_datasloader_N, device)
#     print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
                                         
    H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          test_H_datasloader, device)
    print(H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc)

#     R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           test_R_datasloader, device)
#     print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    
    f1_val_all=int(f1_val_A)+int(f1_val_B)+int(f1_val_C)+int(f1_val_N)+int(f1_val_L)
    
    if int(f1_val_all)>max_val_f1_all:
        max_val_f1_all=int(f1_val_all)
        max_epoch=epoch
        max_model_dict=student_model.state_dict()
        # 指定你想要保存模型参数的路径  
        torch.save(max_model_dict, os.path.join(mdst_path, mdst_name))
        # 使用torch.save()保存模型参数  
        
        max_test_H_dict['auc']=H_auc
        max_test_H_dict['pre']=H_pre
        max_test_H_dict['rec']=H_rec
        max_test_H_dict['f1_test']=H_f1_test
        max_test_H_dict['f1_test_normal']=H_f1_test_normal
        max_test_H_dict['f1_test_AF']=H_f1_test_AF
        max_test_H_dict['f1_test_Other']=H_f1_test_Other
        max_test_H_dict['acc']=H_acc
        
    print(f1_val_all)
        
print('max_epoch=',max_epoch)
print('max_val_f1_all=',max_val_f1_all)
        
print (max_val_f1_all,max_epoch)
print('_____________H_____________::')
print(list(max_test_H_dict.values()))
print('___________________________::')                                         



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'H',
        max_test_H_dict['auc'],
        max_test_H_dict['pre'],
        max_test_H_dict['rec'],
        max_test_H_dict['f1_test'],
        max_test_H_dict['f1_test_normal'],
        max_test_H_dict['f1_test_AF'],
        max_test_H_dict['f1_test_Other'],
        max_test_H_dict['acc']+'\t',
         ],
        ]

with open('log/proposed_ABCLN_H_zishiying_600000_20240321_total_f1.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
    

base_model=cnn_Ag
base_model=cnn_Ag_teacher_self_distill_leadI
ok
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [43:41<00:00,  5.30s/it]

epoch= 0
train_running_loss= 0.5731050976141226


epoch= 0
val_loss= 0.7968953251838684
________________________________
848 646 706 600 445 700 654 581
epoch= 0
val_loss= 0.7279904051260515
________________________________
882 680 757 650 468 781 702 653
epoch= 0
val_loss= 0.585635614891847
________________________________
913 784 821 762 644 903 740 720
epoch= 0
val_loss= 0.6040572699378518
________________________________
906 731 747 701 569 805 729 700
epoch= 0
val_loss= 0.4674198528130849
________________________________
866 721 728 724 897 806 467 822
epoch= 0
test_loss= 0.6101057128150864
________________________________
819 684 662 656 767 646 557 690
3437
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [45:00<00:00,  5.46s/it]

...

epoch= 28
train_running_loss= 0.3928291383333126


epoch= 28
val_loss= 0.4541604624075048
________________________________
868 717 717 717 519 753 878 809
epoch= 28
val_loss= 0.42733701250769873
________________________________
919 736 759 746 504 866 869 818
epoch= 28
val_loss= 0.27792809686313075
________________________________
962 877 861 867 724 976 903 869
epoch= 28
val_loss= 0.2539331255590214
________________________________
978 915 872 892 840 905 930 910
epoch= 28
val_loss= 0.3553857058286667
________________________________
930 793 821 805 919 893 602 865
epoch= 28
test_loss= 0.6172174306789248
________________________________
889 750 737 701 523 907 672 620
4027
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [42:53<00:00,  5.20s/it]

epoch= 29
train_running_loss= 0.36563167891545084


epoch= 29
val_loss= 0.49546000010827007
________________________________
885 736 789 755 610 776 878 817
epoch= 29
val_loss= 0.49065253138542175
________________________________
925 742 817 763 574 864 852 806
epoch= 29
val_loss= 0.28869469743222
________________________________
962 875 900 886 775 980 902 876
epoch= 29
val_loss= 0.2173573103021173
________________________________
985 942 899 919 896 915 946 931
epoch= 29
val_loss= 0.34316415226820746
________________________________
933 801 830 813 920 901 616 869
epoch= 29
test_loss= 0.5219258437357327
________________________________
894 784 797 785 726 909 720 728
4136
max_epoch= 29
max_val_f1_all= 4136
4136 29
_____________H_____________::
['894', '784', '797', '785', '726', '909', '720', '728']
___________________________::


In [8]:
# 更改加载的教师模型字典：
# 加载5项不同的教师模型
# 开始训练流程
Teacher_queue=TeacherQueue(max_size=CRL_queue_len)
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model=generate_model(base_model=base_model_teacher,input_channels=12, num_classes=class_num,DG_method=DG_method_teacher,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model.load_state_dict(torch.load('/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/1711094236_ABCLN_H_proposed_cnnAg_zong_f1.pth'))
teacher_model.eval()
print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)

mdst_path='/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/'
mdst_name=str(int(time.time()))+'_ABCLN_H_zishiying_proposed_xuesheng_cnnAg_zong_f1.pth'

max_val_f1_all=0
max_epoch=0
max_test_H_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}
# max_test_R_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}

for epoch in range(total_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    train_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_datasloader,device,GRL_loss_beta,GAN_aug='0211',GAN_model=cls_model)
    
    auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_A, device)
    print(auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc)
    auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_B, device)
    print(auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc)
    auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_C, device)
    print(auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc)
    auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_N, device)
    print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
    auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_L, device)
    print(auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc)
#     auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           val_datasloader_N, device)
#     print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
                                         
    H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          test_H_datasloader, device)
    print(H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc)

#     R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           test_R_datasloader, device)
#     print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    
    f1_val_all=int(f1_val_A)+int(f1_val_B)+int(f1_val_C)+int(f1_val_N)+int(f1_val_L)
    
    if int(f1_val_all)>max_val_f1_all:
        max_val_f1_all=int(f1_val_all)
        max_epoch=epoch
        max_model_dict=student_model.state_dict()
        # 指定你想要保存模型参数的路径  
        torch.save(max_model_dict, os.path.join(mdst_path, mdst_name))
        # 使用torch.save()保存模型参数  
        
        max_test_H_dict['auc']=H_auc
        max_test_H_dict['pre']=H_pre
        max_test_H_dict['rec']=H_rec
        max_test_H_dict['f1_test']=H_f1_test
        max_test_H_dict['f1_test_normal']=H_f1_test_normal
        max_test_H_dict['f1_test_AF']=H_f1_test_AF
        max_test_H_dict['f1_test_Other']=H_f1_test_Other
        max_test_H_dict['acc']=H_acc
        
    print(f1_val_all)
        
print('max_epoch=',max_epoch)
print('max_val_f1_all=',max_val_f1_all)
        
print (max_val_f1_all,max_epoch)
print('_____________H_____________::')
print(list(max_test_H_dict.values()))
print('___________________________::')                                         



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'H',
        max_test_H_dict['auc'],
        max_test_H_dict['pre'],
        max_test_H_dict['rec'],
        max_test_H_dict['f1_test'],
        max_test_H_dict['f1_test_normal'],
        max_test_H_dict['f1_test_AF'],
        max_test_H_dict['f1_test_Other'],
        max_test_H_dict['acc']+'\t',
         ],
        ]

with open('log/proposed_ABCLN_H_zishiying_600000_20240321_total_f1.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
    

base_model=cnn_Ag
base_model=cnn_Ag_teacher_self_distill_leadI
ok
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [42:50<00:00,  5.19s/it]

epoch= 0
train_running_loss= 0.5313279812962253


epoch= 0
val_loss= 0.6829633151783663
________________________________
875 624 780 640 515 661 744 669
epoch= 0
val_loss= 0.7267152883789756
________________________________
889 670 796 669 485 804 717 677
epoch= 0
val_loss= 0.5162777031461397
________________________________
933 769 858 779 679 883 774 753
epoch= 0
val_loss= 0.29606813455329223
________________________________
968 878 887 882 857 871 919 898
epoch= 0
val_loss= 0.4922094281875726
________________________________
898 717 790 747 892 825 523 821
epoch= 0
test_loss= 0.5251798938111504
________________________________
874 749 765 750 806 760 685 758
3717
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [44:55<00:00,  5.45s/it]

...

epoch= 28
train_running_loss= 0.36311035399136754


epoch= 28
val_loss= 0.4316262637867647
________________________________
885 738 766 749 592 771 883 821
epoch= 28
val_loss= 0.4147391806949269
________________________________
927 756 807 774 586 865 871 826
epoch= 28
val_loss= 0.27691813341031474
________________________________
962 875 892 882 770 973 902 875
epoch= 28
val_loss= 0.2385078414398081
________________________________
980 929 887 906 867 913 939 922
epoch= 28
val_loss= 0.36956593213659344
________________________________
921 793 836 811 914 910 608 862
epoch= 28
test_loss= 0.5404226360934796
________________________________
904 798 799 789 727 907 735 736
4122
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [52:59<00:00,  6.42s/it]

epoch= 29
train_running_loss= 0.4029159977600343


epoch= 29
val_loss= 0.4783608071944293
________________________________
882 753 745 742 576 776 874 809
epoch= 29
val_loss= 0.47687105699019
________________________________
923 747 795 760 563 860 859 810
epoch= 29
val_loss= 0.28780442103743553
________________________________
963 880 888 883 774 971 904 877
epoch= 29
val_loss= 0.25464343849350424
________________________________
980 919 892 903 877 895 937 918
epoch= 29
val_loss= 0.4247997493454904
________________________________
924 778 803 779 892 889 555 828
epoch= 29
test_loss= 0.5283454542998041
________________________________
881 799 776 787 756 894 710 739
4067
max_epoch= 19
max_val_f1_all= 4125
4125 19
_____________H_____________::
['902', '809', '833', '820', '785', '926', '749', '773']
___________________________::


In [9]:
# 更改加载的教师模型字典：
# 加载5项不同的教师模型
# 开始训练流程
Teacher_queue=TeacherQueue(max_size=CRL_queue_len)
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model=generate_model(base_model=base_model_teacher,input_channels=12, num_classes=class_num,DG_method=DG_method_teacher,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model.load_state_dict(torch.load('/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/1711092851_ABCLN_H_proposed_cnnAg_zong_f1.pth'))
teacher_model.eval()
print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)

mdst_path='/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/'
mdst_name=str(int(time.time()))+'_ABCLN_H_zishiying_proposed_xuesheng_cnnAg_zong_f1.pth'

max_val_f1_all=0
max_epoch=0
max_test_H_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}
# max_test_R_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}

for epoch in range(total_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    train_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_datasloader,device,GRL_loss_beta,GAN_aug='0211',GAN_model=cls_model)
    
    auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_A, device)
    print(auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc)
    auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_B, device)
    print(auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc)
    auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_C, device)
    print(auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc)
    auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_N, device)
    print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
    auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_L, device)
    print(auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc)
#     auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           val_datasloader_N, device)
#     print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
                                         
    H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          test_H_datasloader, device)
    print(H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc)

#     R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           test_R_datasloader, device)
#     print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    
    f1_val_all=int(f1_val_A)+int(f1_val_B)+int(f1_val_C)+int(f1_val_N)+int(f1_val_L)
    
    if int(f1_val_all)>max_val_f1_all:
        max_val_f1_all=int(f1_val_all)
        max_epoch=epoch
        max_model_dict=student_model.state_dict()
        # 指定你想要保存模型参数的路径  
        torch.save(max_model_dict, os.path.join(mdst_path, mdst_name))
        # 使用torch.save()保存模型参数  
        
        max_test_H_dict['auc']=H_auc
        max_test_H_dict['pre']=H_pre
        max_test_H_dict['rec']=H_rec
        max_test_H_dict['f1_test']=H_f1_test
        max_test_H_dict['f1_test_normal']=H_f1_test_normal
        max_test_H_dict['f1_test_AF']=H_f1_test_AF
        max_test_H_dict['f1_test_Other']=H_f1_test_Other
        max_test_H_dict['acc']=H_acc
        
    print(f1_val_all)
        
print('max_epoch=',max_epoch)
print('max_val_f1_all=',max_val_f1_all)
        
print (max_val_f1_all,max_epoch)
print('_____________H_____________::')
print(list(max_test_H_dict.values()))
print('___________________________::')                                         



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'H',
        max_test_H_dict['auc'],
        max_test_H_dict['pre'],
        max_test_H_dict['rec'],
        max_test_H_dict['f1_test'],
        max_test_H_dict['f1_test_normal'],
        max_test_H_dict['f1_test_AF'],
        max_test_H_dict['f1_test_Other'],
        max_test_H_dict['acc']+'\t',
         ],
        ]

with open('log/proposed_ABCLN_H_zishiying_600000_20240321_total_f1.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
    

base_model=cnn_Ag
base_model=cnn_Ag_teacher_self_distill_leadI
ok
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [51:27<00:00,  6.24s/it]

epoch= 0
train_running_loss= 0.5448775227047273


epoch= 0
val_loss= 0.48357677985640135
________________________________
859 678 694 682 526 663 857 783
epoch= 0
val_loss= 0.5214140090075406
________________________________
897 708 777 722 521 827 820 766
epoch= 0
val_loss= 0.36727265765269596
________________________________
938 832 828 828 703 916 867 828
epoch= 0
val_loss= 0.37340833860285144
________________________________
968 889 807 841 786 833 902 871
epoch= 0
val_loss= 0.9342816399805474
________________________________
899 717 751 667 737 837 427 652
epoch= 0
test_loss= 0.6295038873311316
________________________________
868 757 698 716 681 780 687 686
3740
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [47:34<00:00,  5.77s/it]

...

epoch= 2
val_loss= 0.47048768050530376
________________________________
884 750 752 732 614 717 865 801
epoch= 2
val_loss= 0.5168036439202048
________________________________
906 739 776 727 547 813 821 767
epoch= 2
val_loss= 0.36344506156941253
________________________________
950 842 856 837 745 907 860 830
epoch= 2
val_loss= 0.40058181040427265
________________________________
964 909 800 839 842 772 903 870
epoch= 2
val_loss= 0.6152540228583596
________________________________
904 722 749 704 835 809 468 747
epoch= 2
test_loss= 0.5532562478934184
________________________________
899 791 712 736 709 779 720 716
3839
_____________________________________
 


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 495/495 [47:43<00:00,  5.79s/it]

epoch= 3
train_running_loss= 0.5071567573291444


epoch= 3
val_loss= 0.4699901149553411
________________________________
878 732 784 742 581 791 853 787
epoch= 3
val_loss= 0.473095872185447
________________________________
908 722 803 741 545 848 829 780
epoch= 3
val_loss= 0.33575219195336103
________________________________
949 848 877 855 741 952 870 840
epoch= 3
val_loss= 0.2990909420392093
________________________________
972 905 867 884 843 884 924 902
epoch= 3
val_loss= 0.5368923833875945
________________________________
881 727 750 724 858 851 463 779
epoch= 3
test_loss= 0.5396371076602747
________________________________
891 753 809 775 746 852 725 739
3946
_____________________________________
 


 49%|██████████████████████████████████████████████▌                                               | 245/495 [23:10<25:50,  6.20s/it]

In [ ]:
# 更改加载的教师模型字典：
# 加载5项不同的教师模型
# 开始训练流程
Teacher_queue=TeacherQueue(max_size=CRL_queue_len)
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model=generate_model(base_model=base_model_teacher,input_channels=12, num_classes=class_num,DG_method=DG_method_teacher,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model.load_state_dict(torch.load('/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/1711091430_ABCLN_H_proposed_cnnAg_zong_f1.pth'))
teacher_model.eval()
print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)

mdst_path='/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/'
mdst_name=str(int(time.time()))+'_ABCLN_H_zishiying_proposed_xuesheng_cnnAg_zong_f1.pth'

max_val_f1_all=0
max_epoch=0
max_test_H_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}
# max_test_R_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}

for epoch in range(total_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    train_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_datasloader,device,GRL_loss_beta,GAN_aug='0211',GAN_model=cls_model)
    
    auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_A, device)
    print(auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc)
    auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_B, device)
    print(auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc)
    auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_C, device)
    print(auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc)
    auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_N, device)
    print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
    auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_L, device)
    print(auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc)
#     auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           val_datasloader_N, device)
#     print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
                                         
    H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          test_H_datasloader, device)
    print(H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc)

#     R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           test_R_datasloader, device)
#     print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    
    f1_val_all=int(f1_val_A)+int(f1_val_B)+int(f1_val_C)+int(f1_val_N)+int(f1_val_L)
    
    if int(f1_val_all)>max_val_f1_all:
        max_val_f1_all=int(f1_val_all)
        max_epoch=epoch
        max_model_dict=student_model.state_dict()
        # 指定你想要保存模型参数的路径  
        torch.save(max_model_dict, os.path.join(mdst_path, mdst_name))
        # 使用torch.save()保存模型参数  
        
        max_test_H_dict['auc']=H_auc
        max_test_H_dict['pre']=H_pre
        max_test_H_dict['rec']=H_rec
        max_test_H_dict['f1_test']=H_f1_test
        max_test_H_dict['f1_test_normal']=H_f1_test_normal
        max_test_H_dict['f1_test_AF']=H_f1_test_AF
        max_test_H_dict['f1_test_Other']=H_f1_test_Other
        max_test_H_dict['acc']=H_acc
        
    print(f1_val_all)
        
print('max_epoch=',max_epoch)
print('max_val_f1_all=',max_val_f1_all)
        
print (max_val_f1_all,max_epoch)
print('_____________H_____________::')
print(list(max_test_H_dict.values()))
print('___________________________::')                                         



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'H',
        max_test_H_dict['auc'],
        max_test_H_dict['pre'],
        max_test_H_dict['rec'],
        max_test_H_dict['f1_test'],
        max_test_H_dict['f1_test_normal'],
        max_test_H_dict['f1_test_AF'],
        max_test_H_dict['f1_test_Other'],
        max_test_H_dict['acc']+'\t',
         ],
        ]

with open('log/proposed_ABCLN_H_zishiying_600000_20240321_total_f1.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
    

In [ ]:
# 更改加载的教师模型字典：
# 加载5项不同的教师模型
# 开始训练流程
Teacher_queue=TeacherQueue(max_size=CRL_queue_len)
student_model=generate_model(base_model=base_model_student,input_channels=2, num_classes=class_num,DG_method=DG_method_student,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model=generate_model(base_model=base_model_teacher,input_channels=12, num_classes=class_num,DG_method=DG_method_teacher,domain_classes=len(train_data_list),distill=distill).to(device)
teacher_model.load_state_dict(torch.load('/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/1711089958_ABCLN_H_proposed_cnnAg_zong_f1.pth'))
teacher_model.eval()
print('ok')
optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=student_model.parameters(),SAM_flag=False,weight_decay=1e-4)

mdst_path='/home/yangli/AF_DG_OODG/save_model_states/ABCLN_H_/'
mdst_name=str(int(time.time()))+'_ABCLN_H_zishiying_proposed_xuesheng_cnnAg_zong_f1.pth'

max_val_f1_all=0
max_epoch=0
max_test_H_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}
# max_test_R_dict={'auc':0,'pre':0,'rec':0,'f1_test':0,'f1_test_normal':0,'f1_test_AF':0,'f1_test_Other':0,'acc':0}

for epoch in range(total_epoch):
    print('_____________________________________')
    print(' ')
    time.sleep(0.1)
    train_student_one_epoch(epoch,warm_epoch,total_epoch,student_model,teacher_model, \
                            max_lr,min_lr,loss_fn_clf,loss_fn_tslb,optimizer, \
                            feature_contrastive_loss_fn, \
                            Teacher_queue,base_optimizer, \
                            train_datasloader,device,GRL_loss_beta,GAN_aug='0211',GAN_model=cls_model)
    
    auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_A, device)
    print(auc,pre,rec,f1_val_A,f1_val_normal,f1_val_AF_A,f1_val_Other,acc)
    auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_B, device)
    print(auc,pre,rec,f1_val_B,f1_val_normal,f1_val_AF_B,f1_val_Other,acc)
    auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_C, device)
    print(auc,pre,rec,f1_val_C,f1_val_normal,f1_val_AF_C,f1_val_Other,acc)
    auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_N, device)
    print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
    auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          val_datasloader_L, device)
    print(auc,pre,rec,f1_val_L,f1_val_normal,f1_val_AF_L,f1_val_Other,acc)
#     auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc=val_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           val_datasloader_N, device)
#     print(auc,pre,rec,f1_val_N,f1_val_normal,f1_val_AF_N,f1_val_Other,acc)
                                         
    H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
                          test_H_datasloader, device)
    print(H_auc,H_pre,H_rec,H_f1_test,H_f1_test_normal,H_f1_test_AF,H_f1_test_Other,H_acc)

#     R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc=test_student_one_epoch(epoch,warm_epoch, total_epoch, student_model,loss_fn_clf, \
#                           test_R_datasloader, device)
#     print(R_auc,R_pre,R_rec,R_f1_test,R_f1_test_normal,R_f1_test_AF,R_f1_test_Other,R_acc)
    
    f1_val_all=int(f1_val_A)+int(f1_val_B)+int(f1_val_C)+int(f1_val_N)+int(f1_val_L)
    
    if int(f1_val_all)>max_val_f1_all:
        max_val_f1_all=int(f1_val_all)
        max_epoch=epoch
        max_model_dict=student_model.state_dict()
        # 指定你想要保存模型参数的路径  
        torch.save(max_model_dict, os.path.join(mdst_path, mdst_name))
        # 使用torch.save()保存模型参数  
        
        max_test_H_dict['auc']=H_auc
        max_test_H_dict['pre']=H_pre
        max_test_H_dict['rec']=H_rec
        max_test_H_dict['f1_test']=H_f1_test
        max_test_H_dict['f1_test_normal']=H_f1_test_normal
        max_test_H_dict['f1_test_AF']=H_f1_test_AF
        max_test_H_dict['f1_test_Other']=H_f1_test_Other
        max_test_H_dict['acc']=H_acc
        
    print(f1_val_all)
        
print('max_epoch=',max_epoch)
print('max_val_f1_all=',max_val_f1_all)
        
print (max_val_f1_all,max_epoch)
print('_____________H_____________::')
print(list(max_test_H_dict.values()))
print('___________________________::')                                         



# 写入csv

header=[
        'train\\test\t',
        'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]
#         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

        # round

results=[
        [
        'H',
        max_test_H_dict['auc'],
        max_test_H_dict['pre'],
        max_test_H_dict['rec'],
        max_test_H_dict['f1_test'],
        max_test_H_dict['f1_test_normal'],
        max_test_H_dict['f1_test_AF'],
        max_test_H_dict['f1_test_Other'],
        max_test_H_dict['acc']+'\t',
         ],
        ]

with open('log/proposed_ABCLN_H_zishiying_600000_20240321_total_f1.csv','a',encoding='utf-8')as file_obj:
    writer=csv.writer(file_obj)
    writer.writerow(header)
    for result in results:
        writer.writerow(result)
        
    